# Quantization

In [ ]:
import torch
from torch.autograd.function import InplaceFunction, Function
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
import math
import numpy as np
import torch.nn.init as init
import torchvision
from tqdm import tqdm
import json
from pathlib import Path

In [ ]:
def quantize_model(model, quantize = False, bits = 8, qmode = "dynamic"):
    if quantize:
        print("Quantize mode on")
        for layer in model.modules():
            try:
                mode = layer.mode()
                if mode == False:
                    layer.change_mod(True, bits, qmode)
            except:
                continue
    else:
        print("Quantize mode off")
        for layer in model.modules():
            try:
                mode = layer.mode()
                if mode == True:
                    layer.change_mod(False, 0)
            except:
                continue
    return model

In [ ]:
def qsin_activation_mode(model):
    print("QSIN activation mode on")
    for layer in model.modules():
        try:
            mode = layer.mode()
            layer.qsinmode()
        except:
            continue
    return model

In [ ]:
class MyRound(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        return torch.round(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

In [ ]:
def Quantize_tensor(input_tensor, max_abs_val = None, num_bits = 8):
    my_round = MyRound.apply
    qmin = -1.0 * (2**num_bits) / 2
    qmax = -qmin - 1
    scale = max_abs_val / ((qmax - qmin) / 2)
    input_tensor = torch.div(input_tensor, scale)
    input_tensor = my_round((input_tensor))
    input_tensor = torch.clamp(input_tensor, qmin, qmax)
    return torch.mul(input_tensor, scale)

In [ ]:
class Quant(nn.Module):
    def __init__(self, num_bits=8, mode = "dynamic", static_count = 30):
        super(Quant, self).__init__()
        self.num_bits = num_bits
        self.mode = mode
        self.static_count = static_count
        self.static_cur = 0
        self.stat_values = []
        self.max_abs = 0 
        if mode != "dynamic":
            self.max_abs_tr = nn.Parameter(torch.zeros(0), requires_grad=True) # IMPORTANT
        
    def forward(self, input):
        if self.mode == "dynamic":
            self.max_abs = torch.max(torch.abs(input.detach()))
            return Quantize_tensor(input, self.max_abs, self.num_bits)
        
        elif self.mode == "static":
            if self.static_cur > self.static_count:
                return Quantize_tensor(input, self.max_abs_tr, self.num_bits)
            elif self.static_cur == self.static_count:
                self.max_abs = np.mean(self.stat_values)
                self.max_abs_tr.data = torch.tensor(self.max_abs, dtype=torch.float).to(self.max_abs_tr.device)
                self.static_cur += 1
                return Quantize_tensor(input, self.max_abs_tr, self.num_bits)
            else:
                self.static_cur += 1
                self.stat_values.append(np.max(np.absolute(input.cpu().detach().numpy())))
                return input

In [ ]:
def QSin(x, num_bits = 8): 
    pi = torch.tensor(np.pi)
    qmin = -1.0 * (2**num_bits) / 2
    qmax = -qmin - 1
    result = torch.sum(torch.square(torch.sin(torch.mul(pi, x[torch.logical_and(x >= qmin, x <= qmax)]))))
    result = result + torch.sum(torch.mul(torch.square(pi), torch.square((x[x < qmin] - qmin))))
    result = result + torch.sum(torch.mul(torch.square(pi), torch.square((x[x > qmax] - qmax))))
    return result

In [ ]:
class Linear(nn.Linear):
    def __init__(self, in_features: int, out_features: int, bias: bool = True, quantization: bool = False, q_bits: int = 8, qsin_activation = False):
        super(Linear, self).__init__(in_features, out_features, bias)

        self.quantize = True if quantization else False
        self.QsinA = True if qsin_activation else False

        if self.quantize:
            self.bits = q_bits
            self.Quantize_weights = Quant(self.bits)
            self.Quantize_input = Quant(8) 
        else:
            self.bits = 'FP'
            
        if self.QsinA:
            self.qsin_loss_A = 0

    def init(self, input):
        self.inputW = input.shape
        
    def change_mod(self, value, bits = 8, mode = "dynamic"):
        self.quantize = value
        self.bits = bits
        self.Quantize_weights = Quant(bits, mode)
        self.Quantize_input = Quant(8, mode)
        
    def qsinmode(self):
        self.QsinA = True
        self.qsin_loss_A = 0
        
    def mode(self):
        return self.quantize  

    def forward(self, input):
            
        if self.quantize:
            qinput = self.Quantize_input(input)
            qweight = self.Quantize_weights(self.weight)
            
            #count qsin loss on activation
            if self.QsinA:
                self.qsin_loss_A = 0
                qmin = -1.0 * (2**8) / 2
                qmax = -qmin - 1
                scale = self.Quantize_input.max_abs_tr / ((qmax - qmin) / 2)
                sq_scale = torch.square(scale)
                self.qsin_loss_A = torch.mul(sq_scale, QSin(torch.div(input, scale), 8))
            
            return nn.functional.linear(qinput, qweight, self.bias)
        else:
            return nn.functional.linear(input, self.weight, self.bias)

In [ ]:
class Embedding(nn.Embedding):
    def __init__(self, num_embeddings, embedding_dim, padding_idx=None, max_norm=None,
                 norm_type=2.0, scale_grad_by_freq=False, sparse=False,
                 quantization: bool = False, q_bits: int = 8):
        super(Embedding, self).__init__(num_embeddings, embedding_dim, padding_idx)

        self.quantize = True if quantization else False

        if self.quantize:
            self.bits = q_bits
            self.Quantize_weights = Quant(self.bits)
        else:
            self.bits = 'FP'

    def init(self, input):
        self.inputW = input.shape
        
    def change_mod(self, value, bits = 8, mode = "dynamic"):
        self.quantize = value
        self.bits = bits
        self.Quantize_weights = Quant(bits, mode)
        
    def mode(self):
        return self.quantize  

    def forward(self, input):
            
        if self.quantize:
            qweight = self.Quantize_weights(self.weight)
        
            return nn.functional.embedding(input, qweight, self.padding_idx, self.max_norm,
                 self.norm_type, self.scale_grad_by_freq, self.sparse)
        else:
            return nn.functional.embedding(input, self.weight, self.padding_idx, self.max_norm,
                 self.norm_type, self.scale_grad_by_freq, self.sparse)
        

In [ ]:
def Qsin_W(model, bits = 8):
    qmin = -1.0 * (2**bits) / 2
    qmax = -qmin - 1
    loss = 0
    for layer in model.modules():
        try:
            scale = layer.Quantize_weights.max_abs_tr / ((qmax - qmin) / 2)
            sq_scale = torch.square(scale)
            QSin_w = QSin(torch.div(layer.weight, scale), bits)
            loss = loss + torch.mul(sq_scale, QSin_w)
        except:
            continue
    return loss

In [ ]:
def Qsin_A(model):
    loss = 0
    for layer in model.modules():
        try:
            loss = loss + layer.qsin_loss_A
        except:
            continue
    return loss

In [ ]:
def get_custom_Linear(in_features, out_features, bias, weight):
    linear = Linear(in_features, out_features)
    linear.bias = bias
    linear.weight = weight
    return linear

def get_custom_Embeding(num_embeddings, embedding_dim, padding_idx, weight):
    embedding = Embedding(num_embeddings, embedding_dim, padding_idx)
    embedding.weight = weight
    return embedding

def change_layers(model):
    for name, layer in model.named_children():
        #if name == 'intermediate' or \
        #name == 'output'or name == 'embeddings':
        #   continue
        if isinstance(layer, nn.Linear):
            setattr(model, name, get_custom_Linear(
                                                layer.in_features,
                                                layer.out_features,
                                                layer.bias,
                                                layer.weight
            ))
            
        if isinstance(layer, nn.Embedding):
            setattr(model, name, get_custom_Embeding(
                                                layer.num_embeddings,
                                                layer.embedding_dim,
                                                layer.padding_idx,
                                                layer.weight
            ))
        change_layers(getattr(model, name))

In [ ]:
#pooler
#classifier
#attention

In [ ]:
def quantize_model(model, quantize = False, bits = 8, qmode = "dynamic"):
    if quantize:
        print("Quantize mode on")
        for name, layer in model.named_modules():
            try:
                mode = layer.mode()
                if mode == False:
                    if 'classifier' in name: continue 
                    else: layer.change_mod(True, 8, qmode)
            except:
                continue
    else:
        print("Quantize mode off")
        for layer in model.modules():
            try:
                mode = layer.mode()
                if mode == True:
                    layer.change_mod(False, 0)
            except:
                continue
    return model

In [ ]:
#pooler
#classifier
#attention

        #if name == 'intermediate' or \
        #name == 'output'or name == 'embeddings':
        #    continue

# glue metric

In [ ]:
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import f1_score, matthews_corrcoef

import datasets


_CITATION = """\
@inproceedings{wang2019glue,
  title={{GLUE}: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding},
  author={Wang, Alex and Singh, Amanpreet and Michael, Julian and Hill, Felix and Levy, Omer and Bowman, Samuel R.},
  note={In the Proceedings of ICLR.},
  year={2019}
}
"""

_DESCRIPTION = """\
GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.
"""

_KWARGS_DESCRIPTION = """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:
    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'accuracy': 1.0}
    >>> glue_metric = datasets.load_metric('glue', 'mrpc')  # 'mrpc' or 'qqp'
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'accuracy': 1.0, 'f1': 1.0}
    >>> glue_metric = datasets.load_metric('glue', 'stsb')
    >>> references = [0., 1., 2., 3., 4., 5.]
    >>> predictions = [0., 1., 2., 3., 4., 5.]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print({"pearson": round(results["pearson"], 2), "spearmanr": round(results["spearmanr"], 2)})
    {'pearson': 1.0, 'spearmanr': 1.0}
    >>> glue_metric = datasets.load_metric('glue', 'cola')
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'matthews_correlation': 1.0}
"""


def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "accuracy": acc,
        "f1": f1,
    }


def pearson_and_spearman(preds, labels):
    pearson_corr = pearsonr(preds, labels)[0]
    spearman_corr = spearmanr(preds, labels)[0]
    return {
        "pearson": pearson_corr,
        "spearmanr": spearman_corr,
    }


@datasets.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
class Glue(datasets.Metric):
    def _info(self):
        if self.config_name not in [
            "sst2",
            "mnli",
            "mnli_mismatched",
            "mnli_matched",
            "cola",
            "stsb",
            "mrpc",
            "qqp",
            "qnli",
            "rte",
            "wnli",
            "hans",
        ]:
            raise KeyError(
                "You should supply a configuration name selected in "
                '["sst2", "mnli", "mnli_mismatched", "mnli_matched", '
                '"cola", "stsb", "mrpc", "qqp", "qnli", "rte", "wnli", "hans"]'
            )
        return datasets.MetricInfo(
            description=_DESCRIPTION,
            citation=_CITATION,
            inputs_description=_KWARGS_DESCRIPTION,
            features=datasets.Features(
                {
                    "predictions": datasets.Value("int64" if self.config_name != "stsb" else "float32"),
                    "references": datasets.Value("int64" if self.config_name != "stsb" else "float32"),
                }
            ),
            codebase_urls=[],
            reference_urls=[],
            format="numpy",
        )

    def _compute(self, predictions, references):
        if self.config_name == "cola":
            return {"matthews_correlation": matthews_corrcoef(references, predictions)}
        elif self.config_name == "stsb":
            return pearson_and_spearman(predictions, references)
        elif self.config_name in ["mrpc", "qqp"]:
            return acc_and_f1(predictions, references)
        elif self.config_name in ["sst2", "mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]:
            return {"accuracy": simple_accuracy(predictions, references)}
        else:
            raise KeyError(
                "You should supply a configuration name selected in "
                '["sst2", "mnli", "mnli_mismatched", "mnli_matched", '
                '"cola", "stsb", "mrpc", "qqp", "qnli", "rte", "wnli", "hans"]'
            )

# BERT

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./models/model-bert-base/', local_files_only=True)

In [ ]:
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizer
    
batch_size = 32

tokenizer = DistilBertTokenizer.from_pretrained('./models/tokenizer-bert-base/', local_files_only=True)

In [ ]:
from datasets import load_from_disk
encoded_dataset = load_from_disk('cur_glue_data')


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
task = 'cola'
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
metric = metric = Glue(task)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)



In [ ]:
change_layers(model)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

In [ ]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./test-glue/checkpoint-1070/',
                                                           local_files_only=True)

In [ ]:
trainer.evaluate()

In [ ]:
change_layers(model)

In [ ]:
model = quantize_model(model, quantize=False, bits = 8)
model = quantize_model(model, quantize=True, bits = 8)

In [ ]:
Eval_DQ = Trainer(
    model,
    args,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
Eval_DQ.evaluate()

In [ ]:
args = TrainingArguments(
    "test-glue",
    per_device_eval_batch_size=16,
    metric_for_best_model=metric_name,
    disable_tqdm = True,
)

In [ ]:
from tqdm import tqdm

def low_bit_search(model, trainer_args, metric_name, max_drop_level = 40, min_bits = 2):
    trainer = Trainer(model, 
                      trainer_args,  
                      eval_dataset=encoded_dataset[validation_key],
                      tokenizer=tokenizer, compute_metrics=compute_metrics
                    )
    best_quality = trainer.evaluate()['eval_'+metric_name]
    cur_quality = best_quality
    target_quality = best_quality - best_quality * max_drop_level / 100
    for name, layer in tqdm(model.named_modules()):
        try:
            mode = layer.mode()
            if mode == True:
                while cur_quality >= target_quality:
                    num_bits = layer.Quantize_weights.num_bits
                    if num_bits == min_bits:
                        break
                    num_bits -= 1
                    layer.change_mod(True, num_bits, "dynamic")
                    trainer = Trainer(model, args, 
                                      eval_dataset=encoded_dataset[validation_key],
                                      tokenizer=tokenizer, compute_metrics=compute_metrics
                                     )
                    tmp_quality = trainer.evaluate()['eval_'+metric_name]
                    if tmp_quality < target_quality:
                        num_bits += 1
                        layer.change_mod(True, num_bits, "dynamic")
                        break
                    cur_quality = tmp_quality
                        
        except:
            continue
            
    return cur_quality
    

In [ ]:
low_bit_search(model, args, metric_name)

In [ ]:
all_num_bits = []

word_embeddings = 23440896
position_embeddings = 393216
token_type_embeddings = 1536

attention_self_query_weight = 589824
intermediate_dense_weight = 2359296
output_dense_weight = 2359296

p_dense_weight = 589824

sz = 0

for name, layer in tqdm(model.named_modules()):
    try:
        mode = layer.mode()
        if mode == True:
            num_bits = layer.Quantize_weights.num_bits
            all_num_bits.append(num_bits)
            #print("Layer:",name, "| bit:",num_bits)
            
            if 'word_embeddings' in name:
                sz += word_embeddings * num_bits
            elif 'position_embeddings' in name:
                sz += position_embeddings * num_bits
            elif 'token_type_embeddings' in name:
                sz += token_type_embeddings * num_bits
            elif 'attention' in name:
                sz += attention_self_query_weight * num_bits
            elif 'intermediate' in name:
                sz += intermediate_dense_weight * num_bits
            elif 'output.dense' in name:
                sz += output_dense_weight * num_bits
            elif 'bert.pooler.dense' in name:
                sz += p_dense_weight * num_bits
                
    except:
        continue

In [ ]:
sz += 768 * 32 * 9 + 3072 * 32
437.82041599999997 / (sz / 8 * 1e-6)

In [ ]:
#all_num_bits

In [ ]:
def new_static_quantize(model, all_num_bits):
    i = 0
    for name, layer in tqdm(model.named_modules()):
        try:
            mode = layer.mode()
            if mode == True:
                num_bits = all_num_bits[i]
                i += 1
                layer.change_mod(True, num_bits, "dynamic")
                        
        except:
            continue
    return model
    

In [ ]:
for i in range(20):
    model = new_static_quantize(model, all_num_bits)

    trainer = Trainer(model, 
                      args,  
                      eval_dataset=encoded_dataset[validation_key],
                      tokenizer=tokenizer, compute_metrics=compute_metrics
                    )

    trainer.evaluate()
    np.random.shuffle(all_num_bits)

In [ ]:
def new_static_quantize(model, all_num_bits):
    i = 0
    for name, layer in tqdm(model.named_modules()):
        try:
            mode = layer.mode()
            if mode == True:
                num_bits = all_num_bits[i]
                i += 1
                layer.change_mod(True, num_bits, "static")
                        
        except:
            continue
    return model
    

In [ ]:
model = new_static_quantize(model, all_num_bits)
encoded_dataset_static = load_from_disk('cur_glue_data_st')

In [ ]:
train_enc = encoded_dataset_static['train']

In [ ]:
from torch.utils.data import DataLoader
train_enc.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
train_loader = torch.utils.data.DataLoader(train_enc, batch_size=8, shuffle = True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()
i = 0
for batch in tqdm(train_loader):
    i += 1
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    if i == 32:
        break

In [ ]:
args = TrainingArguments(
    "test-glue",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    metric_for_best_model=metric_name,
)

Eval_SQ = Trainer(
    model,
    args,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
Eval_SQ.evaluate()

# QSIN

In [ ]:
def Qsin_W(model):
    loss = 0
    for layer in model.modules():
        try:
            bits = layer.Quantize_weights.num_bits
            qmin = -1.0 * (2**bits) / 2
            qmax = -qmin - 1
            scale = layer.Quantize_weights.max_abs_tr / ((qmax - qmin) / 2)
            sq_scale = torch.square(scale)
            QSin_w = QSin(torch.div(layer.weight, scale), bits)
            loss = loss + torch.mul(sq_scale, QSin_w)
        except:
            continue
    return loss

In [ ]:
qsin_activation_mode(model)
print()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from transformers import Trainer

class QSinTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        Qsin_W_loss = Qsin_W(model)
        Qsin_A_loss = Qsin_A(model)
        L = outputs[0]
        lambda_w = 10 ** (np.round(np.log10(Qsin_W_loss.cuda().tolist()) - np.log10(L.cuda().tolist())))
        lambda_a = 10 ** (np.round(np.log10(Qsin_A_loss.cuda().tolist()) - np.log10(L.cuda().tolist()))+1)
        loss = L + Qsin_W_loss / lambda_w + Qsin_A_loss / lambda_a
        return (loss, outputs) if return_outputs else loss

In [ ]:
args = TrainingArguments(
    "qsin_train_tmp",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    #eval_steps=10, 
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

trainer_QSin = QSinTrainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer_QSin.train()

# QAT

In [ ]:
args = TrainingArguments(
    "qat_train_tmp",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    #eval_steps=10,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)


trainer_QAT = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer_QAT.train()